In [2]:
# Ensure necessary library is installed
install.packages("rvest")
install.packages("tidyverse")
install.packages("lubridate")

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)



In [3]:
# Loading Libraries
library(rvest)
library(tidyverse)
library(lubridate)

In [4]:
#Scrape Data from the Web Page
url <- "https://rateyourmusic.com/charts/top/album/all-time/"
webpage <- read_html(url)

In [5]:
# Extract CSS Selectors
album_names <- webpage %>% 
  html_nodes(".page_charts_section_charts_item_title") %>% 
  html_text() %>%
  str_trim() # Remove any leading or trailing white spaces
artists <- webpage %>% 
  html_nodes(".page_charts_section_charts_item_credited_links_primary") %>% 
  html_text() %>%
  str_trim() # Remove any leading or trailing white spaces
ratings <- webpage %>%
  html_nodes(".page_charts_section_charts_item_details_average_num") %>%
  html_text(trim = TRUE) %>%
  as.numeric() # convert to numeric
ratings <- ratings[seq(1, length(ratings), 2)]
release_date <- webpage %>% 
  html_nodes(".page_charts_section_charts_item_date span:first-child") %>% 
  html_text() %>%
  str_trim() %>% # Remove any leading or trailing white spaces
  dmy() %>%  # parse as day-month-year
  format("%d-%m-%Y")  # format as dd-mm-yyyy
genres <- webpage %>% 
  html_nodes(".page_charts_section_charts_item_genres_primary") %>% 
  map_chr(~.x %>% # This function from the purrr package is used to apply the nested function to each node, and ensures that the results are returned as character vectors.
            html_nodes("a") %>% 
            html_text() %>% 
            paste(collapse = ", ") #This results in a single string for each album that lists all associated genres in one go.
  )
num_ratings <- webpage %>% 
  html_nodes(".page_charts_section_charts_item_details_ratings .full") %>% 
  html_text() %>%
  str_trim() %>% # Remove any leading or trailing white spaces
  str_remove_all(",") %>%  # remove commas from numbers
  as.numeric() # convert to numeric
num_views <- webpage %>% 
  html_nodes(".page_charts_section_charts_item_details_reviews .full") %>% 
  html_text() %>%
  str_trim() %>%
  as.numeric() # convert to numeric


Warning message:
“ 2 failed to parse.”
Warning message in webpage %>% html_nodes(".page_charts_section_charts_item_details_reviews .full") %>% :
“NAs introduced by coercion”


In [6]:
# Transform into a data frame
df <- data.frame(
  Album = album_names,
  Artist = artists,
  Rating = ratings,
  ReleaseDate = release_date,
  Genres = genres,
  NumberOfRatings = num_ratings,
  NumberOfViews = num_views
)
# Adding a ranking column
df <- df %>% mutate(Ranking = row_number())
# To ensure the Ranking column is first
df <- df %>% select(Ranking, everything())
head(df)


,Ranking,Album,Artist,Rating,ReleaseDate,Genres,NumberOfRatings,NumberOfViews
,<int>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
1,1,To Pimp a Butterfly,Kendrick Lamar,4.36,15-03-2015,"Conscious Hip Hop, West Coast Hip Hop, Jazz Rap",69408,597
2,2,OK Computer,Radiohead,4.27,16-06-1997,"Alternative Rock, Art Rock",94474,NA
3,3,Wish You Were Here,Pink Floyd,4.32,12-09-1975,"Progressive Rock, Art Rock",65095,997
4,4,Madvillainy,Madvillain,4.33,23-03-2004,Abstract Hip Hop,54144,469
5,5,In the Court of the Crimson King,King Crimson,4.31,10-10-1969,"Progressive Rock, Art Rock",59831,939
6,6,In Rainbows,Radiohead,4.28,10-10-2007,"Art Rock, Alternative Rock",68354,872
